# Satellite Link Analysis tool

In [1]:
import math
import numpy as np

## Let's define a few constants

In [97]:
EARTH_RADIUS = 6378 # km
GEO_RADIUS = 42174 # km, distance between geostationary satellite and center of the EARTH
GEO_ALTITUDE = GEO_RADIUS - EARTH_RADIUS

## Let's define utility funtions

In [98]:
'''Let's define utility functions'''

def to_deg(rad):
    return rad*180/math.pi

def to_rad(deg):
    return deg*math.pi/180

def to_dB(decimal):
    return 10*math.log10(decimal)

def to_decimal(dB):
    return 10**(dB/10)

def cos_deg(deg):
    return math.cos(to_rad(deg))

def sin_deg(deg):
    return math.sin(to_rad(deg))

## Now let's define getter functions for single quantities

In [99]:
def get_alpha_beta(elevation_angle):
    """
    Compute the angle between the (satellite-earth center) line and the (satellite-earth surface) line.
    The latter has an elevation_angle given with respect to the earth tangent.
    All angles abre computed in degrees
    """
    
    alpha = to_deg(math.asin(EARTH_RADIUS * sin_deg(90. + elevation_angle) / GEO_RADIUS))
    beta = 180. - alpha - 90. - elevation_angle
    return alpha, beta
    

In [100]:
def get_range(elevation_angle, sat_altitude=GEO_ALTITUDE):
    '''
    Computes the range (km) of a satellite with a given elevation angle.
    It is possible to define a sat_altitude in case the satellite is not geostationary'''
    
    alpha, beta = get_alpha_beta(elevation_angle)
    return math.sqrt( (sat_altitude+EARTH_RADIUS)**2 + EARTH_RADIUS**2 - 2*EARTH_RADIUS*(EARTH_RADIUS+sat_altitude) * cos_deg(180-(90+elevation_angle)-alpha)) 

In [101]:
def get_covered_area(elevation_angle, sat_altitude=GEO_ALTITUDE):
    
    '''
    Computes the approximation of the covered area (km^2)of a satellite with a given elevation angle
    '''
    
    alpha, beta = get_alpha_beta(elevation_angle)
    
    if (alpha < 10.):
        the_range = get_range(elevation_angle, sat_altitude)
        r = sin_deg(alpha) * the_range
        return math.pi*r*r
    
    raise ValueError("The angle Theta/Alpha is over 10° so the approximation made by this function will be incorrect.")

In [102]:
def get_orbit_period(orbit_radius):
    '''
    Combutes the orbital period of a satellite in hours'''
    result = 24.*math.pow(orbit_radius/GEO_RADIUS, 1.5)
    pretty_print(result)
    return result

def pretty_print(hour):
    h = int(np.floor(hour))
    frac_h = hour-h
    
    minutes = frac_h * 60
    m = int(np.floor(minutes))
    frac_m = minutes - m
    
    seconds = frac_m * 60
    s = int(np.floor(seconds))
    
    h_str = "hours," if h > 1 else "hour,"
    m_str = "minutes," if m > 1 else "minute,"
    s_str = "seconds." if s > 1 else "second."
    
    print("Duration :", h, h_str, m, m_str, s, s_str)

In [103]:
def get_path_loss(wave_len, the_range, dB=True):
    '''By default the result is in dB'''
    
    result = (wave_len/(4.*math.pi*the_range))**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [104]:
def get_anten_area(dish_diameter):
    """Computes the area of a dish antenna given a specified diameter.
    Parameter and output are in meters"""
    return math.pi*dish_diameter*dish_diameter/4.

In [105]:
def get_lambda(transmit_freq, c=3.0*pow(10,8)):
    """Computes the wavelength of a wave with given frequency"""
    return c/transmit_freq

In [106]:
def get_gain_directive(anten_area, wave_len, dB=True):
    """Computes the gain of a directive antenna with given area and knowing the wave length
    of the incoming signal.
    By default the output is in dBi"""
    
    result = 4*math.pi*anten_area/(wave_len*wave_len)
    
    if not dB:
        return result
    
    return to_dB(result)

In [107]:
def get_gain_parabolic(diameter, wavelen, epsilon=0.55, dB= True):
    """Computes the gain of a parabolic dish antenna with given diameter
    and knowing the wave length of the incoming signal.
    By default the output is in dBi"""
    
    result = epsilon * (math.pi*diameter/wavelen)**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [108]:
def get_beamwidth_parabolic(diameter, wavelen, epsilon=0.55, deg=True):
    """Computes the beamwidth of a parabolic dish antenna with a given diameter
    and given the signal wavelength.
    By default the output is in degrees"""
    result = wavelen/(diameter*math.sqrt(epsilon)) 
    
    if not deg:
        return result
    
    return to_deg(result)

In [115]:
def get_EIRP(PT, GT, in_dB=True):
    """Computes the effective isotropic radiated power of a transmitter,
    given its power PT and gain GT. 
    If in_dB== True then PT and GT must be both in dB system, and the output is in dBW.
    Else, PT and GT must be in decimal and the output is in W"""
    
    if in_dB:
        return PT + GT
    
    return PT*GT

In [116]:
def get_Aeff(wavelen, isoG=1.):
    """Computes the effective area with the relative gain to an isotropic radiator"""
    return wavelen*wavelen/(4*math.pi)

In [117]:
def get_flux_density(PR, wavelen, isoG=1., in_dB=True):
    """Computes the flux density with the relative gain to an isotropic radiator.
    If in_dB == True then PR must be in dBW and the output is in dBW/m^2.
    Else, PR must be in W and the output is in W/m^2 """
    
    Aeff = get_Aeff(wavelen, isoG=isoG)
    
    if not in_dB:
        return PR/Aeff
    
    return PR - to_dB(Aeff)

In [118]:
def get_received_power(PT, GT, GR, wavelen, the_range, in_dB=True):
    """Computes the received power for an antenna.
    If input_dB==True then *NOTHING* must be in dB in the parameters and the output is in W
    Else, PT, GT, GR must be in db and the output is in dBW
    The output has the same unit as PT in argument."""
    
    const = (wavelen/(a*math.pi*the_range))**2
    if not in_dB:
        return PT*GT*GR*const # W
    
    return PT + GT + GR + to_dB(const) # dBW
    

In [121]:
def get_C_over_N_alldB(PT, GT, GR,  B, Lp, T= 500, LA=0., LR=0., LM=0.):
    """Computes the ratio C/N.
    Every parameter must be in dB system"""
    
    # Boltzmann
    k = to_dB(1.38*math.pow(10,-23))
    
    return PT+GT+GR+Lp+LA+LR+LM-k-T-B

## Let's define more complete functions to analyze the link

In [126]:
def analysisI(freq,
              trans_diam,
              PT,
              rcv_diam,
              bandw,
              the_range,
              nois_temp,
              loss_atmos,
              loss_rain,
              loss_misc,
              epsilon = 0.55
             ):
    """Here the only parameters in dB are the losses.
    Note also that all distance measures must be in meters"""
    
    # First convert all we need to dB
    nois_temp_dB = to_dB(nois_temp) #dBK
    bandw_dB = to_dB(bandw) #dBHz
    PT_dB = to_dB(PT) #dBW
    
    wavelen = get_lambda(freq)
    print("\nLambda =", wavelen, " m")
    
    Lp = get_path_loss(wavelen, the_range) #dB
    print("\nPath loss =", Lp, "dB")
    
    GT = get_gain_parabolic(trans_diam, wavelen, epsilon=epsilon) #dB
    print("\nTransmitter gain GT =", GT, "dBi")
    
    GR = get_gain_parabolic(rcv_diam, wavelen, epsilon=epsilon) #dB
    print("\nReceiver gain GR =", GR, "dBi")
    
    EIRP = get_EIRP(PT_dB, GT)
    print("\nEIRP =", EIRP, "dbW")
    
    C_N = get_C_over_N_alldB(PT_dB, GT, GR, bandw_dB, Lp, T=nois_temp_dB,
                             LA=loss_atmos, LR=loss_rain, LM=loss_misc)
    
    print("\nC/N =", C_N, "db")
    
    
    return 0

In [130]:
analysisI(2.*pow(10,9), 0.5, 1000, 1.0, pow(10,6), 40000*1000, 500, 3, 3, 6)


Lambda = 0.15  m

Path loss = -190.50357192588757 dB

Transmitter gain GT = 17.804199254431868 dBi

Receiver gain GR = 23.824799167711493 dBi

EIRP = 47.804199254431865 dbW

C/N = 34.73693558888323 db


0